In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sf-booking/hotels_test.csv
/kaggle/input/sf-booking/hotels_train.csv
/kaggle/input/sf-booking/submission.csv


# Конфигурация

In [2]:
# Фиксируем начальное состояние генератора случайных чисе RANDOM_SEED, чтобы эксперимент были воспроизводим! 
RANDOM_SEED = 42

In [3]:
# Директория размещения файлов
DATA_DIR = '/kaggle/input/sf-booking/'

In [4]:
# Фиксируем версию пакетов, чтобы эксперимент были воспроизводим! 
!pip freeze > requirements.txt

# Загрузка данных

In [5]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [6]:
# Объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями



In [7]:
data = pd.concat([df_test, df_train], ignore_index=True)

# Очистка данных

##  Неинформативные признаки
Получение списка неинформативных признаков, у которых доля от общих данных, которую занимает каждое уникальное значение в признаке, более 95%

In [8]:
def get_nunique_ratio(data):
  #список неинформативных признаков
  low_information_cols = [] 

  #цикл по всем столбцам
  for col in data.columns:
      #наибольшая относительная частота в признаке
      top_freq = data[col].value_counts(normalize=True).max()
      #доля уникальных значений от размера признака
      nunique_ratio = data[col].nunique() / data[col].count()
      # сравниваем наибольшую частоту с порогом
      if top_freq > 0.95:
          low_information_cols.append(col)
          print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
      # сравниваем долю уникальных значений с порогом
      if nunique_ratio > 0.95:
          low_information_cols.append(col)
          print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

In [9]:
get_nunique_ratio(data)

## Пропущенные значения

In [10]:
# Получение моды для 'lat'
lat_mode = data['lat'].mode().iloc[0]

# Заполнение пустых значений в 'lat' модой
data['lat'] = data['lat'].fillna(lat_mode)

In [11]:
# Получение моды для 'lng'
lng_mode = data['lng'].mode().iloc[0]

# Заполнение пустых значений в 'lng' модой
data['lng'] = data['lng'].fillna(lng_mode)

## Выбросы (иследование данных по каждому числовому признаку)

In [12]:
def outliers_z_score_mod(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = (data[feature])
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [13]:
# Определяем выбросы

outliers, cleaned = outliers_z_score_mod(
    data = data, feature = 'reviewer_score', log_scale=True, left=3, right=3)
print(f'Число выбросов по методу Z-ОТКЛОНЕНИЙ: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Z-ОТКЛОНЕНИЙ: 0
Результирующее число записей: 515738


# Статистический анализ данных
## Создание новых признаков

В первой версии реализации предсказываем рейтинг только на основе имеющихся числовых признаков

# Отбор признаков

In [14]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 10 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   additional_number_of_scoring                515738 non-null  int64  
 1   average_score                               515738 non-null  float64
 2   review_total_negative_word_counts           515738 non-null  int64  
 3   total_number_of_reviews                     515738 non-null  int64  
 4   review_total_positive_word_counts           515738 non-null  int64  
 5   total_number_of_reviews_reviewer_has_given  515738 non-null  int64  
 6   lat                                         515738 non-null  float64
 7   lng                                         515738 non-null  float64
 8   sample                                      515738 non-null  int64  
 9   reviewer_score                              515738 non-null  float64
d

# Построение модели

In [16]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  

# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

## Дубликаты

In [17]:
# Поиск дублирующих записей
duplicates = train_data[train_data.duplicated()]
duplicates.shape[0]

9090

In [18]:
# Удаление дублирующих записей
train_data.drop_duplicates(inplace=True)

In [19]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)

In [20]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

((128935, 9), (377713, 9), (386803, 8), (290102, 8), (96701, 8))

In [21]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [22]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [23]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.8s finished


In [24]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.14140988421038633


In [25]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [26]:
predict_submission = model.predict(test_data)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.4s finished


In [27]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

,reviewer_score,id
0,8.480000,488440
1,7.675000,274649
2,8.604000,374688
3,9.528000,404352
4,9.389956,451596
5,8.813000,302161
6,7.627000,317079
7,7.691000,13963
8,8.773833,159785
9,7.817000,195089
